In [ ]:
# !pip install tabgan;
# !pip install autogluon;
# !pip install optuna;
# !pip install pytorch-tabnet;

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns
# from datetime import datetime
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, RandomizedSearchCV, GridSearchCV, GroupShuffleSplit
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from catboost import CatBoostRegressor, Pool
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import gc
import lightgbm as lgb
%matplotlib inline
from tqdm.notebook import tqdm
import pickle
import shap
from bayes_opt import BayesianOptimization

# import torch
# from autogluon.tabular import TabularDataset, TabularPredictor
# import optuna
# from optuna import Trial, visualization
# from pytorch_tabnet.tab_model import TabNetRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Hello all!

I'm Zeyneb, a junior at Saratoga High School and the High School Winner of this year's WiDS Datathon! I am also a WiDS Ambassador from the Bay Area. I have experience working with data science and machine learning in projects and research, yet this was a relatively new area of application for me--I usually work with NLP and text. I joined a little later on in the competition, about February. It was really and amazing oppertunity to learn and I was able to apply a lot of new skills and learn new methods working on such and interesting problem to solve real-world problems. I had a lot of fun, and I would love to thank everyone at WiDS for making these events so that people like me can really grow and experience *real* data science! I got to explore a lot of new things working on this, and I wanted to share my approach and findings in the problem! Please don't hesitate to contact me with any questions or comments! 

Data Exploration:
- Right from the start, the data presented some very interesting characteristics. There is a looong time gap between train and test data, a significant differnece in the value distritbutions of the features, and outliers that seems to have comparably more/less temperature change over time. Of course, these can result in some interesting behaviors in the models, explored later below.
- I explored and based some code on the following notebooks (Thank you!):
    - https://www.kaggle.com/code/nicholasdominic/wids2023-data-buddies
    - https://www.kaggle.com/code/kooaslansefat/wids-2023-bo-catboost-xai-pseudo-labeling
    - https://www.kaggle.com/code/kooaslansefat/wids-2023-woman-life-freedom/notebook
    
Interesting/Unintuitive Findings (?):
- In initial experiments, I label encoded features with ~50 distinct values in the train data as categorical features when building the model, then selected top (30%-15%) features based on shapley values. I also performed hyperparameter tuning with 3-5 fold cross validation (split based on location i.e., 'loc_group', or random split). The final output performed relatively poorly (RMSE ~1.5). Strangely , specifying categorical features explicitly into the CatBoost and LightGBM models decreased performance, as did location based splitting. Despite the risks of overfitting and data leakage, random splits and less feature selection seemed to do better. 
- Furthermore, high number of boosting iterations (20K+) with small learning rate also did better 
- I expected overfitting and shakeup in the private leaderboard but I decided to explore and experiment with these.

Experiments that DIDN'T make the final approach:
- Several forecasts for target variable are included in the data. I wanted to use these forecasts as proxy for target. (Strangely, it seems forecasts are constant for the first half of each month). I calculated average daily distance between forecasts and target value from training data (location-day_of_year), and used this as a feature for both train and test. I also added this value to test forecasts to estimate target. 
- I experimented with TabNet and AutoGluon as well for the models. I also worked with some augmentation methods like TabGAN and MixUp to generate synthetic data similar to the test set with pseudolabels. This helped however I have had inconsistent results over multiple runs, so I decided to leave them for replicability.
- Climate Region "Expert" Models: I have also experimented with building a CatBoost model for each climate region with complete training data. I assigned higher sample weights for a that climate region and used that model to predict that specific climate region's test data. For example, for climate region = 'BSk', I assigned sample weights for training data where if the region is BSk then I assigned 1, 0.33 otherwise. I built a CatBoost model and then I used this model to predict only climate region BSk. This also helped improve RMSE.  

Final Approach:
- A key method I used is "iterative pseudolabeling" the test data to incoperate into the training, explained further.
- I primarilly worked with CatBoost and LightGBM models, combining train and pseudo labeled test data, and ensemble the resulting models with previous steps' ensembled model. I iterate over 2 times and finally ensemble with climate region experts. While ensembling, the idea that I used is favoring the most recent model more, and use the ensembled (new and previous step's model) predictions only if the absolute difference between these predictions is below a certain threshold that is set based on experiment. If they differ a lot then use the new model. 
Final Approach Steps:
1. Build LightGBM and CatBoost models, LGB0 and CB0.
2. Build CatBoost Model, CB1, on train data with another set of hyperparameters (2K iterations)
3. Pseudolabel test data from CB1 predictions and build CatBoost model, CB_PL0, with the same hyperparameters from step 2 on train+test data. 
4. Ensemble LGB0, CB0 and CB_PL0 models and use these predictions to pseudolabel test data and build a CatBoost model CB_PL1 with the same CatBoost hyperparameters as Steps 2 and 3, but increase iterations to 25K. Also build LightGBM model, LGB_PL1. Ensemble these two models, also ensemble with the previously ensembled models. i.e., New predictions are from ensembling LGB0, CB0, CB_PL0, LGB_PL1 and CB_PL1. 
5. Use the latest predictions to pseudolabel tets data and build climate expert models. Ensemble climate experts predictions with the latest ensemble to get the final submission. 
- The models run in just over an hour.

## 1. Some EDA

In [ ]:
train_raw = pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv', parse_dates=["startdate"])
test_raw = pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv', parse_dates=["startdate"])
target = 'contest-tmp2m-14d__tmp2m'

test_raw[target] = 0
train_raw['is_train'] = 1
test_raw['is_train'] = 0
all_data = pd.concat([train_raw, test_raw])

del train_raw, test_raw
gc.collect()

In [ ]:
cat_candidates = ['elevation__elevation', 'mjo1d__phase', 'mei__meirank', 'mei__nip', 'loc_group']

# Create Location id
location_group = ['lat', 'lon']
scale = 14

all_data.loc[:,'lat']=round(all_data.lat,scale)
all_data.loc[:,'lon']=round(all_data.lon,scale)

all_data = all_data.sort_values(location_group) 
all_data['loc_group'] = all_data.groupby(location_group, dropna = False).ngroup()
all_data.loc_group.nunique()

all_data = all_data.sort_values('lat') 
all_data['lat_group'] = all_data.groupby('lat', dropna = False).ngroup()
all_data.lat_group.nunique()

all_data = all_data.sort_values('lon') 
all_data['lon_group'] = all_data.groupby('lon', dropna = False).ngroup()
all_data.lat_group.nunique()

all_data['startdate'] = pd.to_datetime(all_data['startdate'])
all_data['year'] = all_data['startdate'].dt.year
all_data['month'] = all_data['startdate'].dt.month
all_data['day_of_year'] = all_data['startdate'].dt.dayofyear

le = LabelEncoder()
all_data['climateregions__climateregion'] = le.fit_transform(all_data['climateregions__climateregion'])

def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

all_data['month'] = all_data.startdate.dt.month
all_data = encode(all_data, 'month', 12)

all_data['day'] = all_data.startdate.dt.day
all_data = encode(all_data, 'day', 31)

all_data = encode(all_data, 'day_of_year', 365)

for c in cat_candidates:
    all_data[c] = all_data[c].astype('int64')

In [ ]:
train = all_data[all_data.is_train == 1]
train = train.reset_index()
test = all_data[all_data.is_train == 0]
test = test.reset_index()
train.shape, test.shape

Features to be used as categorical

In [ ]:
for i in train.columns.tolist():
    if i in ['is_train']:
        continue
    if train[i].nunique() < 50:
        print(i)

In [ ]:
train_na_cols = []
test_na_cols = []
for f in train.columns.tolist():
    if train[f].isna().sum() > 0:
        train_na_cols.append(f)
    if test[f].isna().sum() > 0:
        test_na_cols.append(f)
        
train_na_cols, test_na_cols

Features with na (Test doesn't have any) 

In [ ]:
train.loc_group.nunique(), len([x for x in train.loc_group.unique() if x not in test.loc_group.unique()]), len([x for x in test.loc_group.unique() if x not in train.loc_group.unique()])

There are 514 locations

In [ ]:
all_data.groupby(['is_train', 'year', 'month'])['loc_group'].count().reset_index().sort_values(['is_train', 'year', 'month'])

Test has months 11 and 12

In [ ]:
train.groupby('climateregions__climateregion')['loc_group'].nunique().reset_index().sort_values('loc_group')

Climate regions 0, 5, 12, 13 and 14 have fewer than 10 locations within them

In [ ]:
sns.set(rc={'figure.figsize':(7,5)})
tmp = train[(train.climateregions__climateregion == 1) & (train.year == 2015) & (train.month.isin([11, 12]))]
sns.lineplot(data = tmp, x = 'day_of_year', y= target, hue = 'loc_group')
plt.title('Target over months 11, 12, year 2015, climate region BSk (1)')
plt.show()

In [ ]:
region = 14 #[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
y = 2015
sns.set(rc={'figure.figsize':(10,7)})
forecasts = [ 
    'nmme-tmp2m-34w__cancm3',
 'nmme-tmp2m-34w__cancm4',
 'nmme-tmp2m-34w__ccsm3',
 'nmme-tmp2m-34w__ccsm4',
 'nmme-tmp2m-34w__cfsv2',
 'nmme-tmp2m-34w__gfdl',
 'nmme-tmp2m-34w__gfdlflora',
 'nmme-tmp2m-34w__gfdlflorb',
 'nmme-tmp2m-34w__nasa', 
 'nmme-tmp2m-34w__nmmemean'
]
locs = train[(train.climateregions__climateregion == region) & (train.year == 2015) & (train.month.isin([11, 12]))].loc_group.unique().tolist()[:2]
for l in locs:
    tmp3 = train[(train.loc_group == l) & (train.year == 2015) & (train.month.isin([11, 12]))].copy()
    for f in forecasts:
        sns.lineplot(data = tmp3, x = 'day_of_year', y= f, label = f)
    sns.lineplot(data = tmp3, x = 'day_of_year', y= target, label = '2015 train target', linewidth = 3)
   # plt.title( 'region: '+ str(tmp3.climateregions__climateregion.unique()[0])+  ' loc: '+ str(l))
    plt.legend(loc='lower left', fontsize = 10)
    plt.xlabel(target)
    plt.show()


In [ ]:
tmp = train[(train.month == 12) & (train.year == 2015)][['lat_group', 'lon_group', target]].groupby(['lat_group', 'lon_group'])[target].mean().reset_index()
tmp = tmp.pivot('lon_group','lat_group', target)
tmp.sort_index(level=0, ascending=False, inplace=True)
sns.set(rc={'figure.figsize':(10,7)})
sns.heatmap(tmp)
plt.title('Average target (~temperature) over month 12')
plt.show()

In [ ]:
tmp = train.copy()
tmp = tmp[['lat', 'lon', 'climateregions__climateregion']].drop_duplicates()
sns.set(rc={'figure.figsize':(10,7)})
sns.scatterplot(data = tmp, x = 'lat', y = 'lon', hue = 'climateregions__climateregion', s = 200);

In [ ]:
del train, test, all_data, train_na_cols, test_na_cols, tmp, tmp3, region, y, locs, forecasts
gc.collect()

## 2. Final Model

In [ ]:
train_raw = pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv', parse_dates=["startdate"])
test_raw = pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv', parse_dates=["startdate"])
submit = pd.read_csv('/kaggle/input/widsdatathon2023/sample_solution.csv')
target = 'contest-tmp2m-14d__tmp2m'
fname = '/kaggle/working/'
# models = {}

In [ ]:
def rmse(actual, predicted):
    return mean_squared_error(actual, predicted, squared=False)

In [ ]:
# Train XGBoost and LightGBM models
def location_feature_db(train, test):
    # Reference: https://www.kaggle.com/code/flaviafelicioni/wids-2023-different-locations-train-test-solved
    scale = 14
    train.loc[:,'lat']=round(train.lat,scale)
    train.loc[:,'lon']=round(train.lon,scale)
    test.loc[:,'lat']=round(test.lat,scale)
    test.loc[:,'lon']=round(test.lon,scale)
    
    train_and_test = pd.concat([train, test], axis=0)
    train_and_test['loc_group'] = train_and_test.groupby(['lat', 'lon']).ngroup()
    print(f'{train_and_test.loc_group.nunique()} unique locations')
    
    train = train_and_test.iloc[:len(train)]
    test = train_and_test.iloc[len(train):].drop(target, axis=1)
    
    return train, test

def cat_encode_db(train, test):
    # encoding the categorical feature in the train and test data set
    # using OneHotEncoder
    ohe = OneHotEncoder()
    train_encoded = ohe.fit_transform(train[['climateregions__climateregion']])
    test_encoded = ohe.transform(test[['climateregions__climateregion']])
    
    train = train.drop(['climateregions__climateregion'], axis=1)
    test = test.drop(['climateregions__climateregion'], axis=1)
    
    train_encoded = pd.DataFrame(train_encoded.toarray(), columns=ohe.get_feature_names_out(['climateregions__climateregion']))
    test_encoded = pd.DataFrame(test_encoded.toarray(), columns=ohe.get_feature_names_out(['climateregions__climateregion']))
    
    train = pd.concat([train, train_encoded], axis=1)
    test = pd.concat([test, test_encoded], axis=1)
    
    return train, test
    
def fill_na_rows_db(dataset):
    # Find the columns with missing values
    columns_with_missing_values = dataset.columns[dataset.isnull().any()].tolist()
    
    # Impute the missing values with the mean value of that column
    for col in columns_with_missing_values:
        dataset[col].fillna(dataset[col].mean(), inplace=True)
        
    return dataset

def create_new_feat_db(dataset):
    dataset['year']=pd.DatetimeIndex(dataset['startdate']).year 
    dataset['month']=pd.DatetimeIndex(dataset['startdate']).month 
    dataset['day']=pd.DatetimeIndex(dataset['startdate']).day
    return dataset

def identify_correlated_db(df, threshold):
    corr_matrix = df.corr().abs()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    reduced_corr_matrix = corr_matrix.mask(mask)
    features_to_drop = [c for c in reduced_corr_matrix.columns if any(reduced_corr_matrix[c] > threshold)]
    return features_to_drop

def feature_engineering_db(origin_train, origin_test):
    train, test = origin_train, origin_test
    train = fill_na_rows_db(train)
    train = create_new_feat_db(train)
    test = create_new_feat_db(test)
    train, test = cat_encode_db(train, test)
    irrelevant_cols = ['index', 'startdate', 'climateregions__climateregion', target]
    features = [col for col in train.columns if col not in irrelevant_cols]
    X = train[features]
    X_test = test[features + ['index']]
    y = train[target]
    return X, y, X_test


In [ ]:

X_db, y_db, X_test_db = feature_engineering_db(train_raw.copy(), test_raw.copy())

correlation_threshold = 0.80
features_to_drop = identify_correlated_db(train_raw.copy(), correlation_threshold)
remove_feature = ['index', target]
features_to_drop_v1 = [ele for ele in features_to_drop if ele not in remove_feature]

print(len(features_to_drop_v1),features_to_drop_v1)

X_db.drop(features_to_drop_v1, axis=1, inplace = True)
print("Dropped features that are highly correlated")
X_train, X_val, y_train, y_val = train_test_split(X_db, y_db, test_size=0.33, random_state=42)

In [ ]:
X_train, y_train = X_db.copy(), y_db.copy()
features = X_train.columns.tolist()

lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'max_depth': 8,
    'learning_rate': 0.03,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 50,
    'min_data_in_leaf': 100,
    'subsample_for_bin': 200000,
    'n_estimators': 15000,
    'early_stopping_rounds': 50
    ,'device_type': 'gpu'
}

# Create the LightGBM model object
lgb_model = lgb.LGBMRegressor(**lgb_params)

# Fit the model to the training data
lgb_model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=3000)

# Generate predictions on the validation data
val_pred_lgb = lgb_model.predict(X_val)

# Calculate the RMSE
val_rmse_lgb = rmse(y_val, val_pred_lgb)
print("lgb RMSE:", val_rmse_lgb)

# Make predictions on the competition test data
test_pred_lgb = lgb_model.predict(X_test_db[features])

# models['lgb_model'] = (lgb_model, features)

print("Training and predicting using xgboost")
xgb_params = {'base_score': 0.5, 
          'booster': 'gbtree',
          'tree_method': 'gpu_hist',
          'n_estimators': 15000,
          'objective': 'reg:squarederror',
          'max_depth': 6,
          'subsample': 0.5,
          'colsample_bytree': 0.5,
          'gamma': 1.4,
          'min_child_weight': 7,
          'learning_rate': 0.01,
          'verbose':3000
          ,'gpu_id': 0
          }

xgb_model = xgb.XGBRegressor(**xgb_params)

xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=3000)

# make predictions on the validation data
val_pred_xgb = xgb_model.predict(X_val)

# calculate the RMSE
val_rmse_xgb =rmse(y_val, val_pred_xgb)
print("xgb RMSE:", val_rmse_xgb)


# Make predictions on the competition test data
test_pred_xgb = xgb_model.predict(X_test_db[features])

# models['xgb_model'] = (xgb_model, features)

cb_params = {'iterations': 15000,
          'learning_rate': 0.01,
          'depth': 6,
          'l2_leaf_reg': 3,
          'bagging_temperature': 1,
          'border_count': 256,
          'loss_function': 'RMSE',
          'random_seed': None,
          'task_type': 'GPU',
          'verbose': 3000}

# Create the Catboost model object
cb_model = CatBoostRegressor(**cb_params)

# Fit the model to the training data
cb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=3000)

# Generate predictions on the validation data
val_pred_cb = cb_model.predict(X_val)

# Calculate the RMSE
val_rmse_cb = rmse(y_val, val_pred_cb)
print("lgb RMSE:", val_rmse_cb)

# Make predictions on the competition test data
test_pred_cb = cb_model.predict(X_test_db[features])

# models['cb_model'] = (cb_model, features)

submission_db = X_test_db[['index']].copy()
submission_db['xgb'] = test_pred_xgb
submission_db['lgb'] = test_pred_lgb
submission_db['cb'] = test_pred_cb

submission_db.to_csv(fname+ 'submission_db_0.csv', index = False)
print(fname)

In [ ]:
submission_db.head()

In [ ]:
del X_db, y_db, X_test_db, X_train, X_val, y_train, y_val #, val_pred_xgb, test_pred_xgb, val_pred_lgb, test_pred_lgb,val_pred_cb, test_pred_cb
gc.collect()

In [ ]:
def location_nom_wlf(train, test):
    # Ref: https://www.kaggle.com/code/flaviafelicioni/wids-2023-different-locations-train-test-solved
    scale = 14

    train.loc[:,'lat']=round(train.lat,scale)
    train.loc[:,'lon']=round(train.lon,scale)
    test.loc[:,'lat']=round(test.lat,scale)
    test.loc[:,'lon']=round(test.lon,scale)

    all_df = pd.concat([train, test], axis=0)
    all_df['loc_group'] = all_df.groupby(['lat','lon']).ngroup()
    train = all_df.iloc[:len(train)]
    test = all_df.iloc[len(train):].drop(target, axis=1)
    
    return train, test

def categorical_encode_wlf(train, test):
    le = LabelEncoder()
    train['climateregions__climateregion'] = le.fit_transform(train['climateregions__climateregion'])
    test['climateregions__climateregion'] = le.transform(test['climateregions__climateregion'])
    return train, test
    
def fill_na_wlf(df):
    df = df.sort_values(by=['loc_group', 'startdate']).ffill()
    return df

def creat_new_featute_wlf(df):
    df['year'] = df['startdate'].dt.year
    df['month'] = df['startdate'].dt.month
    df['day_of_year'] = df['startdate'].dt.dayofyear
    return df

def feature_engineering_wlf(train_raw, test_raw):
    train, test = location_nom_wlf(train_raw, test_raw)
    train = fill_na_wlf(train)
    train = creat_new_featute_wlf(train)
    test = creat_new_featute_wlf(test)
    train, test = categorical_encode_wlf(train, test)

    drop_cols = ['index', 'startdate', 'lat', 'lon', target]
    features = [col for col in train.columns if col not in drop_cols]
    X = train[features]
    X_test = test[features + ['index']]
    y = train[target]

    return X, y, X_test

In [ ]:
X, y, X_test = feature_engineering_wlf(train_raw.copy(), test_raw.copy())

In [ ]:
X1, Y1 = X.copy(), y.copy()
features = X1.columns.tolist()
cb_params = {
  'iterations':2000,
  'learning_rate' : 0.0980689972639084,
#   'subsample' : 0.7443133148363695, 
  'l2_leaf_reg' : 2.3722386345448316,
  'max_depth' : int(6.599144674342465),
  'use_best_model' : True, 
  'loss_function' : 'RMSE',
  'model_size_reg' : 0.4833187897595954,
    'task_type': 'GPU',
    'verbose':500
 }

train_pool = Pool(data=X1,label = Y1)
X_train, X_val, y_train, y_val = train_test_split(X1, Y1, test_size=0.33, random_state=42)

cb_model = CatBoostRegressor(**cb_params)
cb_model.fit(train_pool, eval_set=(X_val, y_val), plot=True)

# make predictions on the validation data
val_pred_cb = cb_model.predict(X_val)
# calculate the RMSE
val_rmse_cb =rmse(y_val, val_pred_cb)
print("cb RMSE:", val_rmse_cb)

print(cb_model.get_best_score())

test_pred_cb  = cb_model.predict(X_test[features])
# models['cb_model_0'] = (cb_model, features)

In [ ]:
# Pseudo Labelling: Iteration 0
train_pseudo = X_test[features].copy()
train_pseudo[target] = test_pred_cb
X_train_new = X1
X_train_new[target] = Y1

train_pseudo = pd.concat([X_train_new, train_pseudo], axis=0).reset_index(drop=True)

X_PL0 = train_pseudo[features]
y_PL0 = train_pseudo[target]

del train_pseudo, X_train_new
gc.collect()

# Pseudo Labeling with catboost model only
train_pool = Pool(data=X_PL0, label = y_PL0)

X_train, X_val, y_train, y_val = train_test_split(X_PL0, y_PL0, test_size=0.33, random_state=42)


cb_model_PL0 = CatBoostRegressor(**cb_params)
cb_model_PL0.fit(train_pool, eval_set=(X_val, y_val), plot=True)

# make predictions on the validation data
val_pred_cb_PL0 = cb_model_PL0.predict(X_val)

# calculate the RMSE
val_rmse_cb_PL0 =rmse(y_val, val_pred_cb_PL0)
print("cb RMSE:", val_rmse_cb_PL0)

print(cb_model_PL0.get_best_score())
test_pred_cb_PL0  = cb_model_PL0.predict(X_test[features])

# models['cb_model_PL0'] = (cb_model_PL0, features)

submission_wlf = X_test[['index']].copy()
submission_wlf['cb_PL0'] = test_pred_cb_PL0

In [ ]:
# ensemble WLF catboost with Pseudo labeled model with DB xgboost and lightgbm models
submission_wlf = submission_wlf.merge(submission_db, on = 'index')
# weights= {'lgb':0.27,  'cb':0.48} # 'xgb':0.16,
# weights= {'lgb':0.27, 'cb':0.40, 'cb_PL0':0.33} #  'xgb':0.16,
weights= {'lgb':0.37, 'cb':0.31, 'cb_PL0':0.33} # 'xgb':0.16,
submission_wlf['ensemble'] = (submission_wlf['lgb'] * weights['lgb'] + 
                            submission_wlf['cb'] * weights['cb'] + 
                            submission_wlf['cb_PL0'] * weights['cb_PL0'])

submission_wlf.to_csv(fname+ 'submission_wlf_PL0_1.csv', index = False)

In [ ]:
del (X_PL0, y_PL0, train_pool, X_train, X_val, y_train, y_val, X1, Y1, 
     submission_db, val_pred_cb_PL0, test_pred_cb_PL0,
     val_pred_xgb, test_pred_xgb, val_pred_lgb, test_pred_lgb,val_pred_cb, test_pred_cb)
gc.collect()

In [ ]:
submission_wlf.head()

In [ ]:
# #  Pseudo Labeling Iteration 1: Second Round Catboost model building and Pseudo Labeling with the target created above

X1, Y1 = X.copy(), y.copy()
features = X1.columns.tolist()
cb_params = {
          'iterations':25000,
          'verbose':3000,
          'learning_rate' : 0.0980689972639084,
          #'subsample' : 0.7443133148363695, 
          'l2_leaf_reg' : 2.3722386345448316,
          'max_depth' : int(6.599144674342465),
          'use_best_model' : True, 
          'loss_function' : 'RMSE',
          'model_size_reg' : 0.4833187897595954,
    'task_type': 'GPU',
         }

# Pseudo Labelling
psedolb = submission_wlf[['index', 'ensemble']].copy()
psedolb.rename(columns={'ensemble': target}, inplace=True)

train_pseudo = X_test.copy()
train_pseudo = train_pseudo.merge(psedolb, on = 'index')
train_pseudo = train_pseudo[features + [target]]
X_train_new= X1 
X_train_new[target] = Y1

train_pseudo = pd.concat([X_train_new, train_pseudo], axis=0).reset_index(drop=True)

X_PL1 = train_pseudo[features]
y_PL1 = train_pseudo[target]

del train_pseudo, X_train_new 
gc.collect()

# Pseudo Labeling with catboost model only
train_pool = Pool(data=X_PL1, label = y_PL1)
X_train, X_val, y_train, y_val = train_test_split(X_PL1, y_PL1, test_size=0.33, random_state=42)


cb_model_PL1 = CatBoostRegressor(**cb_params)
cb_model_PL1.fit(train_pool, eval_set=(X_val, y_val), plot=True)

# make predictions on the validation data
val_pred_cb_PL1 = cb_model_PL1.predict(X_val)
# calculate the RMSE
val_rmse_cb_PL1 =rmse(y_val, val_pred_cb_PL1)
print("cb RMSE:", val_rmse_cb_PL1)

print(cb_model_PL1.get_best_score())
test_pred_cb_PL1  = cb_model_PL1.predict(X_test[features])

# models['cb_model_PL1'] = (cb_model_PL1, features)

submission_wlf_pl1 = X_test[['index']].copy()
submission_wlf_pl1['cb_PL1'] = test_pred_cb_PL1

lgb_params = {'boosting_type': 'gbdt',
          'objective': 'regression',
          'metric': 'rmse',
          'max_depth': 4,
          'num_leaves': 31,
          'learning_rate': 0.09,
          'feature_fraction': 0.9,
          'bagging_fraction': 0.8,
          'bagging_freq': 5,
          'early_stopping_round': 50
              ,'device_type': 'gpu',
          'n_estimators': 15000}

lgb_model_PL1 = lgb.LGBMRegressor(**lgb_params)
lgb_model_PL1.fit(X_PL1, y_PL1, eval_set=[(X_val, y_val)], verbose=3000)

# make predictions on the validation data
val_pred_lgb_PL1 = lgb_model_PL1.predict(X_val)
# calculate the RMSE
val_rmse_lgb_PL1 =rmse(y_val, val_pred_lgb_PL1)
print("lgb RMSE:", val_rmse_lgb_PL1)

test_pred_lgb_PL1 = lgb_model_PL1.predict(X_test[features])
submission_wlf_pl1['lgb_PL1'] = test_pred_lgb_PL1

xgb_model_PL1 = xgb.XGBRegressor(base_score=0.5, 
                           n_estimators=20000,
                           objective='reg:linear',
                           max_depth=5,
                           early_stopping_rounds=100,
                           tree_method = 'gpu_hist',
                           gpu_id= 0,
                           learning_rate=0.01)

xgb_model_PL1.fit(X_PL1, y_PL1, eval_set=[(X_val, y_val)], verbose=3000)

# make predictions on the validation data
val_pred_xgb_PL1 = xgb_model_PL1.predict(X_val)
# calculate the RMSE
val_rmse_xgb_PL1 =rmse(y_val, val_pred_xgb_PL1)
print("cb RMSE:", val_rmse_xgb_PL1)

test_pred_xgb_PL1 = xgb_model_PL1.predict(X_test[features])

# models['lgb_model_PL1'] = (lgb_model_PL1, features)

submission_wlf_pl1['xgb_PL1'] = test_pred_xgb_PL1


In [ ]:
del X_train, y_train, train_pool, X1, Y1 , X_PL1, y_PL1
gc.collect()

In [ ]:
# ensemble weights
# weights= {'lgb':0.43, 'xgb':0.07, 'cb':0.58}
weights = {'lgb':0.37,  'cb':0.63} # 'xgb':0.07
submission_wlf_pl1['ensemble'] = (submission_wlf_pl1['lgb_PL1'] * weights['lgb'] +  
                                submission_wlf_pl1['cb_PL1'] * weights['cb']) # submission_wlf_pl1['xgb_PL1'] * weights['xgb']  +
submission_wlf_pl1.to_csv(fname + 'submission_wlf_PL1_2.csv', index = False)

In [ ]:
submission_wlf.head()

In [ ]:
submission_wlf_pl1.head()

In [ ]:
submission = submission_wlf[['index', 'ensemble']].merge(submission_wlf_pl1[['index', 'ensemble']], on = 'index', suffixes = ['_db_PL0', '_PL1'])
weights = {'ensemble_db_PL0':0.15,  'ensemble_PL1':0.84, 'quality_threshold':0.5}
submission['ensemble'] = np.where(abs(submission['ensemble_db_PL0'] - submission['ensemble_PL1']) < weights['quality_threshold'],
                                  submission['ensemble_db_PL0'] * weights['ensemble_db_PL0'] + submission['ensemble_PL1'] * weights['ensemble_PL1'],
                                 submission['ensemble_PL1'])
submission[target] = submission['ensemble']
submission[['index', target]].to_csv(fname+'submission.csv', index = False)

In [ ]:
submission.head()

In [ ]:
# Pseudo Labelling Iteration 2:
# Climate region optimization
from tqdm.notebook import tqdm

X1, Y1 = X.copy(), y.copy()
features = X1.columns.tolist()

psedolb = submission[['index', target]].copy()

train_pseudo = X_test.copy()
train_pseudo = train_pseudo.merge(psedolb, on = 'index')
train_pseudo = train_pseudo[features + [target]]
X_train_new= X1 
X_train_new[target] = Y1

train_pseudo = pd.concat([X_train_new, train_pseudo], axis=0).reset_index(drop=True)

X_PL1 = train_pseudo[features]
y_PL1 = train_pseudo[target]

del train_pseudo, X_train_new 
gc.collect()

cb_params = {'iterations': 10000,
            'learning_rate': 0.05,
            'depth': 4,
            'l2_leaf_reg': 3,
            'bagging_temperature': 0,
            'border_count': 128,
            'loss_function': 'RMSE',
            'random_seed': 42,
            'task_type': 'GPU',
            'verbose': 5000}

climate = X_test[['index', 'climateregions__climateregion']].copy()

for REGION in X_PL1["climateregions__climateregion"].unique().tolist():
    print(REGION)
    weights = X_PL1["climateregions__climateregion"].apply(lambda x: 1 if x==REGION else 0.3)
    train_pool = Pool(data=X_PL1, label = y_PL1, weight = weights)
    cb_model = CatBoostRegressor(**cb_params) 
    cb_model.fit(train_pool, eval_set=(X_val, y_val))
    f_name = target + str(REGION)
    climate[f_name] = cb_model.predict(X_test[features])
    

climate_regions = sorted(X["climateregions__climateregion"].unique().tolist())

climate_map = {climate_regions[i]:i for i in range(len(climate_regions))}

climate_cols =  [target + str(climate_regions[i]) for i in range(len(climate_regions))] + ['index', 'climateregions__climateregion'] 

climate['ensemble'] = climate[climate_cols].apply(lambda x: x[climate_map[x[-1]]], axis = 1)
climate['avg_climate'] = climate[climate_cols].apply(lambda x: np.mean(x), axis = 1)
# climate.to_csv(fname + 'climate_3.csv',index=False)

In [ ]:
climate_df = climate[['index', 'ensemble', 'avg_climate']].merge(submission_wlf_pl1, on = 'index', suffixes = ['_climate', '_db_PL0_PL1'])

# ensemble weights
weights= {'ensemble_climate':0.13, 'ensemble_db_PL0_PL1':0.87, 'quality_threshold': 0.6}
weights= {'ensemble_climate':0.1, 'ensemble_db_PL0_PL1':0.9, 'quality_threshold': 0.6}

climate_df['ensemble'] = np.where(abs(climate_df['ensemble_db_PL0_PL1'] - climate_df['ensemble_climate']) < weights['quality_threshold'],  
                              climate_df['ensemble_db_PL0_PL1']*weights['ensemble_db_PL0_PL1'] + climate_df['ensemble_climate'] * weights['ensemble_climate'],
                              climate_df['ensemble_db_PL0_PL1'])
climate_df.to_csv(fname+ 'submission_wlf_pl1_ensembled_climate_4.csv',index=False)
climate_df[target] = climate_df['ensemble']
climate_df[['index', target]].to_csv('submission_climate.csv', index = False)

In [ ]:
climate_df[['index', target]]